# TempoQL Demo on Full MIMIC-IV Dataset

Before running, please `cd` into this repo and run `pip install .` in the virtual environment of your choice. This will install required dependencies so that you can run this notebook.

This example will use the MIMIC-IV-OMOP dataset available [here](https://physionet.org/content/mimic-iv-demo-omop/0.9/). Please download it and set the path to it in the `base_path` variable below.

In [3]:
from tempo_ql import GenericDataset, formats, QueryEngine, FileVariableStore
import numpy as np
import os
import pandas as pd
import time

/Users/maziyong/Desktop/tempo-ql/tempo_ql/evaluator.py:14: SyntaxWarning: invalid escape sequence '\/'
  GRAMMAR = """


In [2]:
# GCP project in which to run queries - make sure it has access to MIMIC-IV through physionet.org
project_id = "ai-clinician"
# name of a dataset within your project to store temporary results. Required if you plan to subset the data to run queries
scratch_dataset = None
# directory to store temporary variables
variable_store_dir = "mimiciv_data"

# provide your Gemini API key here if you'd like to try the LLM-assisted authoring workflow
gemini_api_key = None

In [3]:
# Initialize query engine and variable store
dataset = GenericDataset(f'bigquery://{project_id}', formats.mimiciv(), 
                        scratch_schema_name=f'{project_id}.{scratch_dataset}' if scratch_dataset is not None else None)

if not os.path.exists(variable_store_dir): os.mkdir(variable_store_dir)
var_store = FileVariableStore(variable_store_dir)
query_engine = QueryEngine(dataset, variable_stores=[var_store])


In [7]:
# get available concepts
query_engine.dataset.list_data_elements(scope="chartevents", return_counts=True).head(20)

,id,name,count,scope,type
1613,227969,Safety Measures,12869833,chartevents,event
1070,220045,Heart Rate,8752069,chartevents,event
2283,220210,Respiratory Rate,8636655,chartevents,event
2031,220277,O2 saturation pulseoxymetry,8567015,chartevents,event
866,220048,Heart Rhythm,7947289,chartevents,event
765,224650,Ectopy Type 1,7049295,chartevents,event
1622,220179,Non Invasive Blood Pressure systolic,5378740,chartevents,event
1924,220180,Non Invasive Blood Pressure diastolic,5377689,chartevents,event
772,220181,Non Invasive Blood Pressure mean,5372922,chartevents,event
171,227958,Less Restrictive Measures,5021996,chartevents,event


In [ ]:
# Get all visit occurrence IDs
query_engine.get_ids()

In [6]:
start = time.time()
result = query_engine.query("first ((time({id in (220181, 220052); scope = chartevents}) where #value < (mean {id in (220181, 220052); scope = chartevents} from #now - 8 h to #now)) - #now) from #now to #now + 24 h every 24 h")
print(len(result), "results,", time.time() - start, "seconds")

ValueError: Exception occurred when processing variable 'None': 'now' keyword can only be used within a time-series expression, ending with an 'at'/'every'/'at every' clause.

In [4]:

import pandas_gbq

# Your existing code should work
df = pandas_gbq.read_gbq("""
    SELECT * FROM `physionet-data.mimiciv_3_1_hosp.admissions`
    LIMIT 100
""", project_id=project_id)

GenericGBQException: Reason: 403 POST https://bigquery.googleapis.com/bigquery/v2/projects/ai-clinician/queries?prettyPrint=false: Access Denied: Project ai-clinician: User does not have bigquery.jobs.create permission in project ai-clinician.

In [2]:
from google.cloud import bigquery
client = bigquery.Client(project="ai-clinician")
print(client.get_service_account_email())


/Users/maziyong/anaconda3/envs/tempo-ql/lib/python3.13/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


bq-934406051607@bigquery-encryption.iam.gserviceaccount.com


In [1]:
import pandas_gbq
project_id = "ai-clinician"
df = pandas_gbq.read_gbq("""
    SELECT * FROM `physionet-data.mimiciv_3_1_hosp.admissions`
    LIMIT 100
""", project_id=project_id)
df

/Users/maziyong/anaconda3/envs/tempo-ql/lib/python3.13/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


GenericGBQException: Reason: 403 POST https://bigquery.googleapis.com/bigquery/v2/projects/ai-clinician/queries?prettyPrint=false: Access Denied: Table physionet-data:mimiciv_3_1_hosp.admissions: User does not have permission to query table physionet-data:mimiciv_3_1_hosp.admissions, or perhaps it does not exist.